In [8]:
import os
from web3 import Web3
from dotenv import load_dotenv
from pathlib import Path
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from bit import wif_to_key
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

import subprocess
import json
from constants import *

In [2]:
import dotenv
import os
dotenv.load_dotenv()

Python-dotenv could not parse statement starting at line 10


True

In [3]:
mnemonic = os.getenv('MNEMONIC')
print(len(mnemonic))

96


In [21]:
def derive_wallets(coin):
    command = f'./derive -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --format=json --coin="{coin}" --numderive= 3'    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    docs = json.loads(output)
    return docs


In [13]:
coins = {ETH: derive_wallets(ETH), 
         BTCTEST: derive_wallets(BTCTEST)}
print(coins)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [18]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [19]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas({
            "from": account.address, 
            "to": to, 
            "value": amount})
        return {
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)}
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [20]:
def send_tx(coin, account, to, amount):
    tx = create_tx(coin, account, to, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH: 
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    elif coin == BTCTEST: 
        return NetworkAPI.broadcast_tx_testnet(signed_tx)